# Spiral Fractal Analysis

Complete analysis pipeline for controlled spiral fractals.

**ISEF Research Project**: Beyond the Mandelbrot: Modeling Real-World Spiral Growth Using Expanding Complex Dynamics

This notebook:
1. Loads sweep results
2. Computes geometry metrics for all spirals
3. Visualizes distributions
4. Explores parameter relationships
5. Identifies stability regions
6. Exports top spirals

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from tqdm.auto import tqdm

# Add repo root to path
ROOT = Path.cwd().parent
sys.path.insert(0, str(ROOT))

from src.geometry import analyze_spiral_image, GeometryConfig

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['figure.dpi'] = 100

print(f"Root: {ROOT}")

## Cell 2: Load Sweep CSV and Filter Usable Images

In [ ]:
# Load sweep results
csv_path = ROOT / "results" / "spiral_sweep.csv"

if not csv_path.exists():
    raise FileNotFoundError(
        f"Sweep CSV not found at {csv_path}.\n"
        "Run: python -m scripts.sweep_params --controlled-only"
    )

df = pd.read_csv(csv_path)
print(f"Loaded {len(df)} rows from sweep CSV")

# Filter to rows with valid image paths
df = df[df['image_path'].notna() & (df['image_path'] != "")].copy()
print(f"Filtered to {len(df)} rows with valid images")

# Add absolute paths
df['image_path_abs'] = df['image_path'].apply(lambda p: str(ROOT / p))

# Verify images exist
df['image_exists'] = df['image_path_abs'].apply(lambda p: Path(p).exists())
n_missing = (~df['image_exists']).sum()
if n_missing > 0:
    print(f"WARNING: {n_missing} images missing on disk")
    df = df[df['image_exists']].copy()

print(f"\nFinal dataset: {len(df)} images")
print(f"\nMap types: {df['map_type'].value_counts().to_dict()}")
if 'radial_mode' in df.columns:
    print(f"Radial modes: {df['radial_mode'].value_counts().to_dict()}")

df.head()

## Cell 3: Compute Geometry Metrics for All Images

In [ ]:
# Create geometry config
cfg = GeometryConfig(
    threshold="otsu",
    min_arm_length=50,
    center=None,
    box_sizes=[2, 4, 8, 16, 32, 64],
    n_subsamples=10,
)

# Compute metrics for each image
metrics_list = []

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Analyzing spirals"):
    img_path = row['image_path_abs']
    
    try:
        metrics = analyze_spiral_image(img_path, cfg)
        metrics['id'] = row['id']
        metrics_list.append(metrics)
    except Exception as e:
        print(f"ERROR analyzing {row['id']}: {e}")
        # Add row with NaNs
        metrics_list.append({
            'id': row['id'],
            'arm_count': np.nan,
            'b_mean': np.nan,
            'b_std': np.nan,
            'r2_mean': np.nan,
            'arm_spacing_mean': np.nan,
            'arm_spacing_std': np.nan,
            'fractal_dimension': np.nan,
            'fractal_dimension_ci_low': np.nan,
            'fractal_dimension_ci_high': np.nan,
        })

# Convert to dataframe
metrics_df = pd.DataFrame(metrics_list)

# Merge with sweep data
full_df = df.merge(metrics_df, on='id', how='left')

# Save metrics
metrics_path = ROOT / "results" / "spiral_metrics.csv"
metrics_path.parent.mkdir(parents=True, exist_ok=True)
full_df.to_csv(metrics_path, index=False)

print(f"\nSaved metrics to {metrics_path}")
print(f"\nMetrics summary:")
print(full_df[['arm_count', 'b_mean', 'r2_mean', 'fractal_dimension']].describe())

full_df.head()

## Cell 4: Basic Distributions

In [ ]:
# Create output directory
fig_dir = ROOT / "figures" / "analysis"
fig_dir.mkdir(parents=True, exist_ok=True)

# Filter to finite values for plotting
valid_df = full_df[full_df['b_mean'].notna() & np.isfinite(full_df['b_mean'])].copy()

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Histogram of b_mean
axes[0, 0].hist(valid_df['b_mean'], bins=30, edgecolor='black', alpha=0.7)
axes[0, 0].set_xlabel('b (log-spiral slope)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].set_title('Distribution of Log-Spiral Slope b')
axes[0, 0].axvline(valid_df['b_mean'].median(), color='red', linestyle='--', label='Median')
axes[0, 0].legend()

# Histogram of arm_count
axes[0, 1].hist(full_df['arm_count'].dropna(), bins=range(int(full_df['arm_count'].max())+2), 
                edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Number of Arms')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('Distribution of Arm Count')

# Histogram of fractal_dimension
fd_valid = full_df['fractal_dimension'].dropna()
fd_valid = fd_valid[np.isfinite(fd_valid)]
if len(fd_valid) > 0:
    axes[1, 0].hist(fd_valid, bins=30, edgecolor='black', alpha=0.7)
    axes[1, 0].set_xlabel('Fractal Dimension')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title('Distribution of Fractal Dimension')

# R^2 distribution
r2_valid = full_df['r2_mean'].dropna()
r2_valid = r2_valid[np.isfinite(r2_valid)]
if len(r2_valid) > 0:
    axes[1, 1].hist(r2_valid, bins=30, edgecolor='black', alpha=0.7)
    axes[1, 1].set_xlabel('R² (fit quality)')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title('Distribution of Log-Spiral Fit Quality')

plt.tight_layout()
plt.savefig(fig_dir / 'distributions.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Saved distributions plot to {fig_dir / 'distributions.png'}")

## Cell 5: Parameter Relationships

In [ ]:
# Filter to controlled spirals with valid metrics
controlled_df = full_df[
    (full_df['map_type'] == 'controlled') &
    full_df['b_mean'].notna() &
    np.isfinite(full_df['b_mean'])
].copy()

print(f"Analyzing {len(controlled_df)} controlled spirals with valid metrics")

# Separate by radial mode
additive_df = controlled_df[controlled_df['radial_mode'] == 'additive'].copy()
power_df = controlled_df[controlled_df['radial_mode'] == 'power'].copy()

fig, axes = plt.subplots(2, 2, figsize=(14, 12))

# 1. delta_r vs b_mean (additive mode)
if len(additive_df) > 0:
    axes[0, 0].scatter(additive_df['delta_r'], additive_df['b_mean'], alpha=0.6)
    axes[0, 0].set_xlabel('δ (additive radial growth)')
    axes[0, 0].set_ylabel('b (log-spiral slope)')
    axes[0, 0].set_title('Additive Mode: δ vs b')
    axes[0, 0].grid(True, alpha=0.3)

# 2. alpha vs b_mean (power mode)
if len(power_df) > 0:
    axes[0, 1].scatter(power_df['alpha'], power_df['b_mean'], alpha=0.6, color='orange')
    axes[0, 1].set_xlabel('α (power radial growth)')
    axes[0, 1].set_ylabel('b (log-spiral slope)')
    axes[0, 1].set_title('Power Mode: α vs b')
    axes[0, 1].grid(True, alpha=0.3)

# 3. omega vs arm_count
axes[1, 0].scatter(controlled_df['omega'], controlled_df['arm_count'], alpha=0.6, color='green')
axes[1, 0].set_xlabel('ω (angular increment)')
axes[1, 0].set_ylabel('Number of Arms')
axes[1, 0].set_title('ω vs Arm Count')
axes[1, 0].grid(True, alpha=0.3)

# 4. omega vs b_mean colored by radial mode
for mode, color in [('additive', 'blue'), ('power', 'orange')]:
    mode_df = controlled_df[controlled_df['radial_mode'] == mode]
    if len(mode_df) > 0:
        axes[1, 1].scatter(mode_df['omega'], mode_df['b_mean'], 
                          alpha=0.6, label=mode, color=color)
axes[1, 1].set_xlabel('ω (angular increment)')
axes[1, 1].set_ylabel('b (log-spiral slope)')
axes[1, 1].set_title('ω vs b (by radial mode)')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(fig_dir / 'parameter_relationships.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Saved parameter relationships to {fig_dir / 'parameter_relationships.png'}")

In [ ]:
# Heatmap: omega × delta_r -> b_mean (additive mode only)
if len(additive_df) > 0:
    pivot = additive_df.pivot_table(
        values='b_mean',
        index='omega',
        columns='delta_r',
        aggfunc='mean'
    )
    
    plt.figure(figsize=(10, 8))
    sns.heatmap(pivot, annot=True, fmt='.3f', cmap='viridis', cbar_kws={'label': 'b_mean'})
    plt.xlabel('δ (additive radial growth)')
    plt.ylabel('ω (angular increment)')
    plt.title('Heatmap: ω × δ → b (additive mode)')
    plt.tight_layout()
    plt.savefig(fig_dir / 'heatmap_omega_delta_b.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"Saved heatmap to {fig_dir / 'heatmap_omega_delta_b.png'}")
else:
    print("No additive mode data for heatmap")

## Cell 6: Stability Region / Phase Diagrams

In [ ]:
# Define "clean spiral" criteria
controlled_df['clean'] = (
    (controlled_df['arm_count'] >= 2) &
    (controlled_df['r2_mean'] >= 0.7) &
    (controlled_df['b_mean'] > 0)
)

n_clean = controlled_df['clean'].sum()
print(f"Clean spirals: {n_clean} / {len(controlled_df)} ({100*n_clean/len(controlled_df):.1f}%)")

# Phase diagrams
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# 1. (delta_r, omega) for additive mode
if len(additive_df) > 0:
    additive_df['clean'] = (
        (additive_df['arm_count'] >= 2) &
        (additive_df['r2_mean'] >= 0.7) &
        (additive_df['b_mean'] > 0)
    )
    
    clean_add = additive_df[additive_df['clean']]
    unclean_add = additive_df[~additive_df['clean']]
    
    axes[0].scatter(unclean_add['delta_r'], unclean_add['omega'], 
                    alpha=0.4, color='red', label='Unclean', s=30)
    axes[0].scatter(clean_add['delta_r'], clean_add['omega'], 
                    alpha=0.7, color='green', label='Clean', s=30)
    axes[0].set_xlabel('δ (additive radial growth)')
    axes[0].set_ylabel('ω (angular increment)')
    axes[0].set_title('Phase Diagram: Additive Mode (δ, ω)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

# 2. (alpha, omega) for power mode
if len(power_df) > 0:
    power_df['clean'] = (
        (power_df['arm_count'] >= 2) &
        (power_df['r2_mean'] >= 0.7) &
        (power_df['b_mean'] > 0)
    )
    
    clean_pow = power_df[power_df['clean']]
    unclean_pow = power_df[~power_df['clean']]
    
    axes[1].scatter(unclean_pow['alpha'], unclean_pow['omega'], 
                    alpha=0.4, color='red', label='Unclean', s=30)
    axes[1].scatter(clean_pow['alpha'], clean_pow['omega'], 
                    alpha=0.7, color='green', label='Clean', s=30)
    axes[1].set_xlabel('α (power radial growth)')
    axes[1].set_ylabel('ω (angular increment)')
    axes[1].set_title('Phase Diagram: Power Mode (α, ω)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(fig_dir / 'phase_diagrams.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Saved phase diagrams to {fig_dir / 'phase_diagrams.png'}")

## Cell 7: Export Top Spirals

In [ ]:
# Sort by quality metrics: r2_mean desc, then arm_count desc
top_df = controlled_df.sort_values(
    by=['r2_mean', 'arm_count'], 
    ascending=[False, False]
).head(20)

# Save top spirals
top_path = ROOT / "results" / "top_spirals.csv"
top_df.to_csv(top_path, index=False)
print(f"Saved top 20 spirals to {top_path}")

print("\nTop spirals:")
print(top_df[['id', 'radial_mode', 'delta_r', 'alpha', 'omega', 'arm_count', 'b_mean', 'r2_mean']].head(10))

# Show thumbnail grid of top spirals
n_show = min(20, len(top_df))
n_cols = 5
n_rows = int(np.ceil(n_show / n_cols))

fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 3*n_rows))
axes = axes.flatten() if n_show > 1 else [axes]

for i, (idx, row) in enumerate(top_df.head(n_show).iterrows()):
    img_path = row['image_path_abs']
    if Path(img_path).exists():
        img = Image.open(img_path)
        axes[i].imshow(img)
        
        mode_short = row['radial_mode'][:3]
        if row['radial_mode'] == 'additive':
            param_str = f"δ={row['delta_r']:.3f}"
        else:
            param_str = f"α={row['alpha']:.2f}"
        
        axes[i].set_title(
            f"{mode_short} {param_str}\nω={row['omega']:.2f} b={row['b_mean']:.3f}\nR²={row['r2_mean']:.2f}",
            fontsize=8
        )
    else:
        axes[i].text(0.5, 0.5, 'Missing', ha='center', va='center')
    
    axes[i].axis('off')

# Hide unused subplots
for i in range(n_show, len(axes)):
    axes[i].axis('off')

plt.suptitle('Top 20 Spirals (by R² and Arm Count)', fontsize=14, y=1.00)
plt.tight_layout()
plt.savefig(fig_dir / 'top_spirals_grid.png', dpi=120, bbox_inches='tight')
plt.show()

print(f"Saved top spirals grid to {fig_dir / 'top_spirals_grid.png'}")